<a href="https://colab.research.google.com/github/313chengpeng/Analysis-of-tweets-about-chatgpt/blob/main/%E4%B8%BB%E9%A2%98%E5%BB%BA%E6%A8%A1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 使用LDA进行主题建模

In [2]:
# 加载云端硬盘

from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [34]:
import pandas as pd
import re
from tqdm.auto import tqdm
from sklearn.decomposition import LatentDirichletAllocation
from sklearn.feature_extraction.text import CountVectorizer

In [4]:
# 导入数据

df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/bigdata/data_precess.csv', encoding='utf8')
# df.head(10)

# 数据清洗函数

In [50]:
#Code by Leon Wolber https://www.kaggle.com/leonwolber/reddit-nlp-topic-modeling-prediction
import nltk
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

def remove_line_breaks(text):
    text = text.replace('\r', ' ').replace('\n', ' ')
    return text

#remove punctuation
def remove_punctuation(text):
    re_replacements = re.compile("__[A-Z]+__")  # such as __NAME__, __LINK__
    re_punctuation = re.compile("[%s]" % re.escape(string.punctuation))
    '''Escape all the characters in pattern except ASCII letters and numbers'''
    tokens = word_tokenize(text)
    tokens_zero_punctuation = []
    for token in tokens:
        if not re_replacements.match(token):
            token = re_punctuation.sub(" ", token)
        tokens_zero_punctuation.append(token)
    return ' '.join(tokens_zero_punctuation)

def remove_special_characters(text):
    text = re.sub('[^a-zA-z0-9\s]', ' ', text)
    return text

def lowercase(text):
    text_low = [token.lower() for token in word_tokenize(text)]
    return ' '.join(text_low)

def remove_stopwords(text):
    custom_stopwords = ['i', 'amp','chang','artificalintellig' ,'tcogfhukqnqz','chatgpt', 'ai', 'https', 'openai' ,'thing' , 'small', 'start' ,'one', 'artifical', 'intellig' ,'you', 'he', 'she', 'it', 'we', 'they', 'am', 'is', 'are', 'was', 'were', 'been', 'have', 'has', 'had', 'do', 'does', 'did', 'can', 'could', 'would', 'should', 'will', 'shall', 'may', 'might', 'must']
    stop = set(custom_stopwords)
    word_tokens = nltk.word_tokenize(text)
    text = " ".join([word for word in word_tokens if word not in stop])
    return text

#remobe one character words
def remove_one_character_words(text):
    '''Remove words from dataset that contain only 1 character'''
    text_high_use = [token for token in word_tokenize(text) if len(token)>1]      
    return ' '.join(text_high_use)   
    
#%%
# Stemming with 'Snowball stemmer" package
def stem(text):
    stemmer = nltk.stem.snowball.SnowballStemmer('english')
    text_stemmed = [stemmer.stem(token) for token in word_tokenize(text)]        
    return ' '.join(text_stemmed)

def lemma(text):
    wordnet_lemmatizer = WordNetLemmatizer()
    word_tokens = nltk.word_tokenize(text)
    text_lemma = " ".join([wordnet_lemmatizer.lemmatize(word) for word in word_tokens])       
    return ' '.join(text_lemma)


#break sentences to individual word list
def sentence_word(text):
    word_tokens = nltk.word_tokenize(text)
    return word_tokens
#break paragraphs to sentence token 
def paragraph_sentence(text):
    sent_token = nltk.sent_tokenize(text)
    return sent_token    


def tokenize(text):
    """Return a list of words in a text."""
    return re.findall(r'\w+', text)

def remove_numbers(text):
    no_nums = re.sub(r'\d+', '', text)
    return ''.join(no_nums)



def clean_text(text):
    _steps = [
    remove_line_breaks,
    remove_one_character_words,
    remove_special_characters,
    lowercase,
    remove_punctuation,
    remove_stopwords,
    stem,
    remove_numbers
]
    for step in _steps:
        text=step(text)
    return text   
#%%

#https://stackoverflow.com/questions/55557004/getting-attributeerror-float-object-has-no-attribute-replace-error-while
#To avoid with tqdm AttributeError: 'float' object has no attribute

df["text"] = df["text"].astype(str)
df["text"] = [x.replace(':',' ') for x in df["text"]]

<>:27: DeprecationWarning: invalid escape sequence \s
<>:27: DeprecationWarning: invalid escape sequence \s
<ipython-input-50-07a4f8a6a64e>:27: DeprecationWarning: invalid escape sequence \s
  text = re.sub('[^a-zA-z0-9\s]', ' ', text)
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [51]:
df['clean_text'] = pd.Series([clean_text(i) for i in tqdm(df['text'])])

df['clean_text'][0]

  0%|          | 0/117021 [00:00<?, ?it/s]

'bardaigoogl cannabiscommun developerstudentclub domain domainnam domainfor web nft dao mmembervil crypto cryptocur cryptolov cryptonew cryptoinvestor great name sale tcoofkpbryj tcocuwrptkntq'

In [53]:
type(df['clean_text'])

pandas.core.series.Series

In [11]:
from sklearn.decomposition import LatentDirichletAllocation
from sklearn.feature_extraction.text import CountVectorizer

# df['clean_text'] = pd.Series([clean_text(i) for i in tqdm(df['text'])])

# 将'text'列转换为列表格式
text_list = df['clean_text'].tolist()

# 将文本列表转换为文本矩阵
vectorizer = CountVectorizer()
text_matrix = vectorizer.fit_transform(text_list)

# # 定义主题数和主题建模模型
# num_topics = 5
# lda_model = LatentDirichletAllocation(n_components=num_topics, random_state=0)

# # 对文本矩阵进行主题建模
# lda_model.fit(text_matrix)

# # 将主题分配到每个文本中
# topic_assignments = lda_model.transform(text_matrix)

# # 将主题分配添加到DataFrame中
# df['topic'] = topic_assignments.argmax(axis=1)

# # 循环遍历每个主题
# for i in range(num_topics):
#     # 获取分配给主题i的所有文本
#     topic_texts = df[df['topic'] == i]['text']
    
#     # 将文本保存到同一文件中
#     with open('topic_{}.txt'.format(i), 'w') as f:
#         for text in topic_texts:
#             f.write(text + '\n')


  0%|          | 0/117021 [00:00<?, ?it/s]

In [15]:
import gensim
from gensim.utils import simple_preprocess
from gensim.models import LdaModel
from gensim.corpora import Dictionary

/usr/local/lib/python3.9/dist-packages/gensim/matutils.py:23: DeprecationWarning: Please use `triu` from the `scipy.linalg` namespace, the `scipy.linalg.special_matrices` namespace is deprecated.
  from scipy.linalg.special_matrices import triu
/usr/local/lib/python3.9/dist-packages/gensim/corpora/dictionary.py:11: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated since Python 3.3, and in 3.10 it will stop working
  from collections import Mapping, defaultdict


In [32]:
df['clean_text'][0]

'bardaigoogl cannabiscommun developerstudentclub domain domainnam domain chatgpt domain domainnam domainforsal web nft ai dao mmembervill crypto cryptocur cryptolov cryptonew cryptoinvestor great name sale https tcoofkpbryj https tcocuwrptkntq'

TypeError: ignored

# 实验新的数据处理方法

In [55]:
import re
import numpy as np
import pandas as pd
from pprint import pprint 
# Gensim 
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel
# spacy for lemmatization
import spacy
# Plotting tools
import pyLDAvis
import pyLDAvis.gensim  # don't skip this
import matplotlib.pyplot as plt
%matplotlib inline

# Enable logging for gensim - optional
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.ERROR)
import warnings
warnings.filterwarnings("ignore",category=DeprecationWarning)
# NLTK Stop words
from nltk.corpus import stopwords
stop_words = stopwords.words('english')
stop_words.extend(['amp','chang','artificalintellig' ,'tcogfhukqnqz','chatgpt', 'ai', 'https', 'openai' ,'thing' , 'small', 'start' ,'one', 'artifical', 'intellig'])


In [56]:
# Import Dataset
df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/bigdata/data_precess.csv', encoding='utf8')


In [65]:
from nltk.stem import PorterStemmer, WordNetLemmatizer
# Convert to list
data = df['text'].tolist()
# Remove Emails
data = [re.sub('\S*@\S*\s?', '', sent) for sent in data]
# Remove new line characters
data = [re.sub('\s+', ' ', sent) for sent in data]
# Remove distracting single quotes
data = [re.sub("\'", "", sent) for sent in data]
# 去除链接和网址
data =  [re.sub(r"http\S+|www\S+", "",sent) for sent in data]
# 去除特殊字符和标点符号
data = [re.sub(r"\S+@\S+", "",sent) for sent in data]
data =  [re.sub(r"[^a-zA-Z0-9\s]+", "", sent) for sent in data]



print(data[:1])

['CannabisCommunity DeveloperStudentClubs domains domainnames domain ChatGPT DOMAINS domainnames domainforsale web3 nft ai dao Mmemberville Crypto Cryptocurency CryptoLover CryptoNews CryptoInvestor Great name for sale at  ']


In [67]:
def sent_to_words(sentences):
    for sentence in sentences:
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))  # deacc=True removes punctuations
data_words = list(sent_to_words(data))
print(data_words[:1])

[['domains', 'domainnames', 'domain', 'chatgpt', 'domains', 'domainnames', 'domainforsale', 'web', 'nft', 'ai', 'dao', 'mmemberville', 'crypto', 'cryptocurency', 'cryptolover', 'cryptonews', 'cryptoinvestor', 'great', 'name', 'for', 'sale', 'at']]


In [69]:
# 创建二元（Bigram）模型和三元（Trigram）模型
# Build the bigram and trigram models
bigram = gensim.models.Phrases(data_words, min_count=5, threshold=100) # higher threshold fewer phrases.
trigram = gensim.models.Phrases(bigram[data_words], threshold=100)  
# Faster way to get a sentence clubbed as a trigram/bigram
bigram_mod = gensim.models.phrases.Phraser(bigram)
trigram_mod = gensim.models.phrases.Phraser(trigram)
# See trigram example
print(trigram_mod[bigram_mod[data_words[0]]])


/usr/local/lib/python3.9/dist-packages/gensim/models/phrases.py:598: UserWarning: For a faster implementation, use the gensim.models.phrases.Phraser class
  warnings.warn("For a faster implementation, use the gensim.models.phrases.Phraser class")


['domains_domainnames', 'domain', 'chatgpt', 'domains_domainnames_domainforsale', 'web', 'nft', 'ai', 'dao', 'mmemberville', 'crypto', 'cryptocurency', 'cryptolover', 'cryptonews', 'cryptoinvestor', 'great', 'name', 'for', 'sale', 'at']


In [71]:
 # Define functions for stopwords, bigrams, trigrams and lemmatization
def remove_stopwords(texts):
    return [[word for word in simple_preprocess(str(doc)) if word not in stop_words] for doc in texts]
def make_bigrams(texts):
    return [bigram_mod[doc] for doc in texts]
def make_trigrams(texts):
    return [trigram_mod[bigram_mod[doc]] for doc in texts]
def lemmatization(texts, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
     """https://spacy.io/api/annotation"""
     texts_out = []
     for sent in texts:
        doc = nlp(" ".join(sent)) 
        texts_out.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
     return texts_out
 
 
 # Remove Stop Words
data_words_nostops = remove_stopwords(data_words)
# Form Bigrams
data_words_bigrams = make_bigrams(data_words_nostops)
# Initialize spacy 'en' model, keeping only tagger component (for efficiency)
# python3 -m spacy download en
nlp = spacy.load('en_core_web_sm', disable=['parser', 'ner'])
# Do lemmatization keeping only noun, adj, vb, adv
data_lemmatized = lemmatization(data_words_bigrams, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV'])
print(data_lemmatized[:1])

[['domains_domainname', 'domain', 'domains_domainname', 'web', 'cryptoinvestor', 'great', 'name', 'sale']]


In [72]:
 # Create Dictionary
id2word = corpora.Dictionary(data_lemmatized)
# Create Corpus
texts = data_lemmatized
 # Term Document Frequency
corpus = [id2word.doc2bow(text) for text in texts]
 # View
print(corpus[:1])

[[(0, 1), (1, 1), (2, 2), (3, 1), (4, 1), (5, 1), (6, 1)]]


In [77]:
# Build LDA model
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=id2word,
                                            num_topics=8, 
                                           random_state=100,
                                           update_every=1,
                                           chunksize=1000,
                                           passes=10,
                                          alpha='auto',
                                           per_word_topics=True)

In [80]:
# Print the Keyword in the 10 topics
print(lda_model.print_topics())
doc_lda = lda_model[corpus]

[(0, '0.032*"generate" + 0.023*"even" + 0.023*"find" + 0.016*"research" + 0.016*"text" + 0.015*"model" + 0.013*"understand" + 0.011*"datum" + 0.011*"twitter" + 0.011*"friend"'), (1, '0.039*"use" + 0.026*"get" + 0.023*"make" + 0.019*"new" + 0.018*"know" + 0.017*"try" + 0.016*"think" + 0.016*"go" + 0.015*"see" + 0.014*"time"'), (2, '0.044*"create" + 0.029*"human" + 0.022*"play" + 0.022*"help" + 0.018*"tweet" + 0.018*"replace" + 0.018*"thing" + 0.017*"content" + 0.014*"wonder" + 0.012*"wrong"'), (3, '0.042*"technology" + 0.033*"potential" + 0.026*"tech" + 0.026*"web" + 0.022*"digital" + 0.018*"machinelearning" + 0.018*"business" + 0.018*"impact" + 0.016*"tedious" + 0.016*"style"'), (4, '0.033*"chatbot" + 0.021*"challenge" + 0.018*"present" + 0.017*"creation" + 0.017*"bot" + 0.016*"break" + 0.015*"llm" + 0.012*"open" + 0.012*"implication" + 0.012*"soon"'), (5, '0.053*"code" + 0.023*"job" + 0.020*"guess" + 0.018*"hour" + 0.017*"last" + 0.014*"user" + 0.013*"language" + 0.012*"train" + 0.011

In [78]:
# Visualize the topics
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(lda_model, corpus, id2word)
vis


/usr/local/lib/python3.9/dist-packages/pyLDAvis/_prepare.py:243: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  default_term_info = default_term_info.sort_values(


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
1      0.440708  0.047156       1        1  33.307414
7     -0.011829 -0.420326       2        1  14.034315
0     -0.057508  0.074816       3        1  11.200102
2     -0.034957  0.072847       4        1  10.374444
5     -0.074325  0.060792       5        1   9.192647
3     -0.089125  0.054009       6        1   7.476328
6     -0.085833  0.055833       7        1   7.450752
4     -0.087131  0.054873       8        1   6.963997, topic_info=            Term          Freq         Total Category  logprob  loglift
35           ask  11852.000000  11852.000000  Default  30.0000  30.0000
187        write  10309.000000  10309.000000  Default  29.0000  29.0000
65           use  13335.000000  13335.000000  Default  28.0000  28.0000
90          code   4985.000000   4985.000000  Default  27.0000  27.0000
349       answer   6010.000000   6010.000000  Default  26.0000  26.0000
...          ...           ...           ...      ...      ...      ...
557    interview    508.110022    509.021234   Topic8  -4.9438   2.6626
2326       allow    460.569921    461.481205   Topic8  -5.0420   2.6624
29    absolutely    447.798711    448.710474   Topic8  -5.0701   2.6624
2904        blow    423.242844    424.154099   Topic8  -5.1265   2.6623
618      history    420.127894    421.039101   Topic8  -5.1339   2.6623

[271 rows x 6 columns], token_table=      Topic      Freq        Term
term                             
86        4  0.998852     ability
29        8  0.998417  absolutely
295       7  0.997342      accord
1551      8  0.999297    accurate
1301      8  0.998177       admit
...     ...       ...         ...
333       1  0.999800        work
98        7  0.996862       worry
187       2  0.999938       write
701       4  0.998439      writer
1246      4  0.999680       wrong

[242 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[2, 8, 1, 3, 6, 4, 7, 5])

In [81]:
 # Compute Perplexity
print('\nPerplexity: ', lda_model.log_perplexity(corpus))  # a measure of how good the model is. lower the better.
# Compute Coherence Score
coherence_model_lda = CoherenceModel(model=lda_model, texts=data_lemmatized, dictionary=id2word, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score: ', coherence_lda)


Perplexity:  -8.590458748973303

Coherence Score:  0.30039911908738903


In [82]:
pyLDAvis.save_html(vis, 'lda_model_vis.html')

# 寻找最优的主题数

In [ ]:
# 创建一个空列表，用于存储每个主题数的困惑度
perplexity_scores = []

# 循环遍历主题数从2到12
for num_topics in range(2, 13):
    lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                               id2word=id2word,
                                               num_topics=num_topics, 
                                               random_state=100,
                                               update_every=1,
                                               chunksize=1000,
                                               passes=10,
                                               alpha='auto',
                                               per_word_topics=True)
    perplexity_scores.append(lda_model.log_perplexity(corpus))

# 使用matplotlib绘制困惑度随主题数变化的图形
plt.plot(range(2, 13), perplexity_scores)
plt.xlabel("Number of topics")
plt.ylabel("Log Perplexity Score")
plt.show()



# LDA结果可视化